# Imports and program variables


In [37]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, MaxPooling1D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
import pandas as pd

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Conv1D
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint


from sklearn.model_selection import train_test_split # to split dataset into train and test
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences
from time_series_dataset_loader import TimeSeriesDatasetLoader


In [2]:
DATASET_PATH = '../src/Features/2secsplit/MFCC_5'

## Setting up train and test datasets

In [3]:
dataset_loader = TimeSeriesDatasetLoader(DATASET_PATH)
X, y = dataset_loader.get_dataset(type_='emotion_type')

In [4]:
X = np.asarray(X)
y = np.asarray(y)

In [28]:
# Completing with zeros
max_len = len(X[0])
for row in X:
        if len(row) > max_len:
            max_len = len(row)

X = pad_sequences(X, maxlen=max_len, padding='post', dtype='float64')

In [29]:
new_dataset = []

for data_instance in X:
    new_dataset_cell = []
    for t_instance in data_instance:
        for value in t_instance[2:]:
            new_dataset_cell.append(value)

    new_dataset.append(new_dataset_cell)

In [38]:
X = np.asarray(new_dataset)

In [40]:
X

array([[  5.488229 ,   1.559822 ,  -1.786611 , ...,   3.844279 ,
         -2.550276 ,  -0.1753159],
       [ -2.899243 ,  -5.710538 ,  -7.743914 , ...,   0.       ,
          0.       ,   0.       ],
       [  8.574527 ,  -0.4827535,   1.567934 , ...,   0.       ,
          0.       ,   0.       ],
       ...,
       [ -1.619469 ,  -2.271107 ,   3.17616  , ...,   2.473269 ,
         -3.67383  ,  -2.100021 ],
       [ -3.986876 ,   9.301771 ,  -7.790942 , ...,   9.606766 ,
        -10.41208  ,   1.189072 ],
       [  0.1078992, -10.08403  , -10.06888  , ...,   0.       ,
          0.       ,   0.       ]])

In [ ]:
''' More Preprocessing '''

In [129]:
from sklearn.preprocessing import RobustScaler

scalers = {}
for i in range(X.shape[1]):
    scalers[i] = RobustScaler()
    X[:, i, :] = scalers[i].fit_transform(X[:, i, :]) 


In [16]:
from sklearn.preprocessing import Normalizer

scalers = {}
for i in range(X.shape[1]):
    scalers[i] = Normalizer()
    X[:, i, :] = scalers[i].fit_transform(X[:, i, :]) 

In [18]:
X[0][0]

array([ 0.        ,  0.        ,  0.28221284,  0.08020835, -0.09187018,
       -0.10788761,  0.62792463,  0.3675004 ,  0.16599244,  0.18636285,
       -0.35420188,  0.19850305, -0.14629544, -0.18601133, -0.21646694,
        0.18571278])

In [ ]:
''' Finishing Preprocessing '''

In [ ]:
# See X and y details
print(X[:2])
print(X.shape)

print(y[:3])
print(y.shape)

In [41]:
# Split the dataset in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)

print("\ny_train:\n")
print(y_train[:3])
print(y_train.shape)

print("\ny_test:\n")
print(y_test[:3])
print(y_test.shape)


X_train:

9.105775999999999
(1364, 504)

X_test:

-0.1714754
(585, 504)

y_train:

[2 1 2]
(1364,)

y_test:

[1 2 2]
(585,)


In [42]:
# Create categorical matrices
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [44]:
num_rows = X[0].shape[0]
num_columns = X[0].shape[1]
num_channels = 1

X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)

IndexError: tuple index out of range

## Building and Compiling Model

In [45]:
model = Sequential()

model.add(Conv1D(128, 5,padding='same', input_shape=(48,1)))
model.add(Activation('sigmoid'))
model.add(Conv1D(64, 5,padding='same'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(32, 5,padding='same',))
model.add(Activation('sigmoid'))
model.add(Conv1D(32, 5,padding='same',))
model.add(Activation('sigmoid'))
model.add(Flatten())
model.add(Dense(3))
model.add(Activation('softmax'))

In [46]:
# top-k category accuracy
import functools
import keras
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',top3_acc])

In [47]:
# Define bath and epochs
batch_size = 256
epochs = 400
checkpoint_file = '../models/model_checkpoints/original_window_2.h5'

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.0000001)
mcp_save = ModelCheckpoint('../models/model_checkpoints/two_split.h5', save_best_only=True, monitor='val_loss', mode='min')
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test), callbacks=[mcp_save, lr_reduce])

ValueError: Error when checking input: expected conv1d_3_input to have 3 dimensions, but got array with shape (1364, 504)

In [25]:
accuracy_list = cnnhistory.history['val_accuracy']
highest_index = cnnhistory.history['val_accuracy'].index(np.sort(cnnhistory.history['val_accuracy'])[-1])
print(cnnhistory.history['val_accuracy'][highest_index])

0.6239316463470459


In [ ]:
# without prepocessing: 0.5863248109817505
# robust preprocessing: 0.6034188270568848
# normalization: 0.6239316463470459

In [ ]:
import matplotlib.pyplot as plt

# Model Summary

model.summary()

# PLT History info
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()